In [1]:
import datetime as dt
import pandas as pd
import requests
from dask.distributed import LocalCluster
from dask.distributed import Client

In [44]:
api = 'http://127.0.0.1:5000/naive'

def worker(api, query):
    start = dt.datetime.now()
    resp = requests.post(api, data={'s': query})
    elapsed = dt.datetime.now() - start
    return elapsed.total_seconds() * 1000, resp.status_code

In [ ]:
c = LocalCluster(4, 16)
client = Client(c)


qs = ['你好'] * 1000

start = dt.datetime.now()
results = []

for s in qs:
    results.append(client.submit(worker, api, s, pure=False))
    
results = client.gather(results)
elapsed = (dt.datetime.now() - start).total_seconds() * 1000

client.close()
c.close()

times = [r for r, _ in results]
status = [1 if r == 200 else 0 for _, r in results]
df = pd.DataFrame({'latency': times, 'status': status})

In [ ]:
df[['latency']].describe()

In [ ]:
print("QPS: %.2f" % (len(qs) / elapsed * 1000))

In [22]:
elapsed

20002.857